In [8]:
import os
import pandas as pd

# Path to the folder with the CSV files
csv_folder = '../tubitakaiagentprojeleriiinverisetleri/'

import pandas as pd

# Load the datasets
try:
    purchase_df = pd.read_csv(csv_folder+'Purchase.csv')
    before_purchase_df = pd.read_csv(csv_folder+'BeforePurchaseDetailsScreen.csv')
    dog_added_df = pd.read_csv(csv_folder+'DogAdded.csv')
    cat_added_df = pd.read_csv(csv_folder+'CatAdded.csv')
    checkout_opened_df = pd.read_csv(csv_folder+'CheckoutPageOpened.csv')
    address_added_df = pd.read_csv(csv_folder+'AddressAdded.csv')
    sign_up_df = pd.read_csv(csv_folder+'SignUpCompleted.csv')
    creditcard_added_df = pd.read_csv(csv_folder+'CreditcardAdded.csv')

    # Inspect each dataframe
    print("Purchase.csv info and head:")
    purchase_df.info()
    purchase_df.head()

    print("\nBeforePurchaseDetailsScreen.csv info and head:")
    before_purchase_df.info()
    before_purchase_df.head()

    print("\nDogAdded.csv info and head:")
    dog_added_df.info()
    dog_added_df.head()

    print("\nCatAdded.csv info and head:")
    cat_added_df.info()
    cat_added_df.head()

    print("\nCheckoutPageOpened.csv info and head:")
    checkout_opened_df.info()
    checkout_opened_df.head()
    
    print("\nAddressAdded.csv info and head:")
    address_added_df.info()
    address_added_df.head()
    
    print("\nSignUpCompleted.csv info and head:")
    sign_up_df.info()
    sign_up_df.head()
    
    print("\nCreditcardAdded.csv info and head:")
    creditcard_added_df.info()
    creditcard_added_df.head()

except FileNotFoundError as e:
    print(f"Error loading files: {e}. Please ensure all files are uploaded.")
except Exception as e:
    print(f"An error occurred: {e}")

Purchase.csv info and head:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28770 entries, 0 to 28769
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   serviceid         28770 non-null  object
 1   ownerid           28768 non-null  object
 2   ordercreatedtime  28770 non-null  object
 3   servicetype       28770 non-null  object
dtypes: object(4)
memory usage: 899.2+ KB

BeforePurchaseDetailsScreen.csv info and head:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36911 entries, 0 to 36910
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           15209 non-null  object
 1   user_id        15209 non-null  object
 2   event_time     15209 non-null  object
 3   serviceType    15209 non-null  object
 4   uuid.1         21702 non-null  object
 5   user_id.1      21702 non-null  object
 6   event_time.1   21702 non-null  object
 7 

In [ ]:
# --- Data Cleaning and Preparation ---

# 1. Clean BeforePurchaseDetailsScreen.csv
# The file has a strange structure with duplicated columns. We need to merge them.
before_purchase_part1 = before_purchase_df[['uuid', 'user_id', 'event_time', 'serviceType']].dropna()
before_purchase_part2 = before_purchase_df[['uuid.1', 'user_id.1', 'event_time.1', 'serviceType.1']].dropna()
before_purchase_part2.columns = ['uuid', 'user_id', 'event_time', 'serviceType']
before_purchase_df_cleaned = pd.concat([before_purchase_part1, before_purchase_part2], ignore_index=True)
before_purchase_df_cleaned.rename(columns={'user_id': 'ownerid'}, inplace=True)

# Corrigir datas para formato misto
for col in ['event_time']:
    before_purchase_df_cleaned[col] = pd.to_datetime(before_purchase_df_cleaned[col], format='mixed', errors='coerce')

# 2. Clean CheckoutPageOpened.csv
checkout_opened_df.rename(columns={'user_id': 'ownerid'}, inplace=True)

if 'event_time' in checkout_opened_df.columns:
    checkout_opened_df['event_time'] = pd.to_datetime(checkout_opened_df['event_time'], format='mixed', errors='coerce')

# 3. Clean Purchase.csv
purchase_df.dropna(subset=['ownerid'], inplace=True)
purchase_df.rename(columns={'servicetype': 'serviceType'}, inplace=True)

if 'event_time' in purchase_df.columns:
    purchase_df['event_time'] = pd.to_datetime(purchase_df['event_time'], format='mixed', errors='coerce')

# 4. Clean and merge DogAdded.csv and CatAdded.csv
dog_added_df['pet_type'] = 'dog'
dog_added_df.rename(columns={'dogid': 'petid'}, inplace=True)

if 'event_time' in dog_added_df.columns:
    dog_added_df['event_time'] = pd.to_datetime(dog_added_df['event_time'], format='mixed', errors='coerce')

cat_added_df['pet_type'] = 'cat'
cat_added_df.rename(columns={'catid': 'petid'}, inplace=True)

if 'event_time' in cat_added_df.columns:
    cat_added_df['event_time'] = pd.to_datetime(cat_added_df['event_time'], format='mixed', errors='coerce')

pets_df = pd.concat([dog_added_df, cat_added_df], ignore_index=True)

# 5. Clean AddressAdded.csv
address_added_df_cleaned = address_added_df.dropna(subset=['ownerid', 'addressid'])

if 'event_time' in address_added_df_cleaned.columns:
    address_added_df_cleaned['event_time'] = pd.to_datetime(address_added_df_cleaned['event_time'], format='mixed', errors='coerce')

# 6. Clean SignUpCompleted.csv
sign_up_df_cleaned = sign_up_df.dropna(subset=['id'])
sign_up_df_cleaned.rename(columns={'id': 'ownerid'}, inplace=True)

if 'event_time' in sign_up_df_cleaned.columns:
    sign_up_df_cleaned['event_time'] = pd.to_datetime(sign_up_df_cleaned['event_time'], format='mixed', errors='coerce')

# 7. Clean CreditcardAdded.csv
creditcard_added_df_cleaned = creditcard_added_df.dropna(subset=['ownerid', 'creditcardid'])

if 'event_time' in creditcard_added_df_cleaned.columns:
    creditcard_added_df_cleaned['event_time'] = pd.to_datetime(creditcard_added_df_cleaned['event_time'], format='mixed', errors='coerce')

# --- Merging DataFrames ---

# Merge checkout and purchase data to create a conversion funnel
# We are considering a conversion if a user who opened the checkout page for a service type made a purchase for the same service type.
# Aggregate checkout and purchase data
checkout_agg = checkout_opened_df.groupby(['ownerid', 'serviceType']).size().reset_index(name='checkout_count')
purchase_agg = purchase_df.groupby(['ownerid', 'serviceType']).size().reset_index(name='purchase_count')

# Merge for conversion analysis
conversion_df = pd.merge(checkout_agg, purchase_agg, on=['ownerid', 'serviceType'], how='left')
conversion_df['purchase_count'].fillna(0, inplace=True)
conversion_df['converted'] = conversion_df['purchase_count'] > 0

# Now, let's say you want to analyze conversion by pet type.
# You can merge the conversion data with pets data.
# First, let's get the pet type for each owner.
owner_pet_types = pets_df.groupby('ownerid')['pet_type'].unique().apply(list).reset_index(name='pet_types')

# Merge conversion data with the pet types
conversion_by_pet_type_df = pd.merge(conversion_df, owner_pet_types, on='ownerid', how='left')



--- Example Analysis: Conversion Funnel ---
Head of the conversion analysis DataFrame:


C:\Users\caioa\AppData\Local\Temp\ipykernel_27096\2688004627.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  conversion_df['purchase_count'].fillna(0, inplace=True)


,ownerid,serviceType,checkout_count,purchase_count,converted,pet_types
0,0011ef23-7b3d-4cd0-8fb2-1b1fa90a6ac6,Sitting,1,0.0,False,NaN
1,005fc863-7c9c-42ce-af56-38fed3c545f3,AdHoc,2,2.0,True,NaN
2,005fc863-7c9c-42ce-af56-38fed3c545f3,Boarding,1,0.0,False,NaN
3,006d7730-5786-4283-a0a0-7d9fa0b7681e,Planned,2,0.0,False,[dog]
4,00863e2c-667f-493e-bde0-d350807a6252,Planned,2,0.0,False,NaN


In [23]:
# --- Analysis and Visualization ---

# 1. Overall Conversion Rate
total_checkouts = len(checkout_agg)
total_conversions = len(conversion_df[conversion_df['converted']])
overall_conversion_rate = (total_conversions / total_checkouts) * 100

print(f"Overall Conversion Rate: {overall_conversion_rate:.2f}%")


Overall Conversion Rate: 43.92%
